In [1]:
import asyncio

In [15]:
import sys
import os
sys.path[:0] = [os.path.join(os.path.abspath(os.path.curdir), "..", "python")]

In [16]:
import proto

In [3]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [4]:
loop = asyncio.get_event_loop()
loop.set_debug(True)

DEBUG:asyncio:Using selector: EpollSelector


In [5]:
cmake_server = proto.CMakeServer(loop, lambda x: print(x))

In [6]:
reader, writer = loop.run_until_complete(asyncio.open_unix_connection("/home/umogslayer/zenith-cmake/pipe"))

In [7]:
async def read_loop():
    while True:
        read_message = await reader.readline()
        print("Message read: len=%d: %s" % (len(read_message), read_message.decode("utf-8")))
        if reader.at_eof():
            await cmake_server.async_stop_execution()
        await cmake_server.async_report_cmake_server_message(read_message)

In [13]:
async def async_send_message_and_print(request):
    async def send_message_callback(message):
        print("Balalaika")
        writer.write(message)
        await writer.drain()
        print("Balalaika2")
    response_iterator = await cmake_server.async_post_cmake_server_request(request_object=request,
                                                                           async_send_message_cb=send_message_callback)
    async for response in response_iterator:
        print(response)

In [14]:
async def handshake():
    request = {
        "type": "handshake",
        "cookie": "temp",
        "protocolVersion": {
            "major": 1,
        },
        "generator": "Ninja",
        "sourceDirectory": "/home/umogslayer/zenith-cmake/zenith-engine/Src/CMakeEntryPoint",
        "buildDirectory": "/home/umogslayer/zenith-cmake/temp_build",
    }
    await async_send_message_and_print(request)
    
async def globalSettings():
    request = {
        "type": "globalSettings",
        "cookie": "temp",
    }
    await async_send_message_and_print(request)
    
async def configure():
    request = {
        "type": "configure",
        "cookie": "temp",
        "cacheArguments": [],
    }
    await async_send_message_and_print(request)


In [10]:
# loop.run_forever()

In [11]:
loop.create_task(read_loop())
loop.run_until_complete(handshake())
# loop.run_forever()

DEBUG:CMakeServer:CMake server hello message: HelloMessage(supported_versions=[SupportedVersion(isExperimental=True, major=1, minor=2)])
DEBUG:CMakeServer:Server message: {'cookie': 'temp', 'inReplyTo': 'handshake', 'type': 'reply'}


Message read: len=1: 

Balalaika
Balalaika2
Message read: len=23: [== "CMake Server" ==[

Message read: len=91: {"supportedProtocolVersions":[{"isExperimental":true,"major":1,"minor":2}],"type":"hello"}

Message read: len=23: ]== "CMake Server" ==]

Message read: len=1: 

Message read: len=23: [== "CMake Server" ==[

Message read: len=57: {"cookie":"temp","inReplyTo":"handshake","type":"reply"}

Message read: len=23: ]== "CMake Server" ==]

{'cookie': 'temp', 'inReplyTo': 'handshake', 'type': 'reply'}


In [15]:
loop.run_until_complete(globalSettings())

DEBUG:asyncio:poll took 0.020 ms: 1 events
DEBUG:CMakeServer:Server message: {'buildDirectory': '/home/umogslayer/zenith-cmake/temp_build', 'capabilities': {'generators': [{'extraGenerators': [], 'name': 'Watcom WMake', 'platformSupport': False, 'toolsetSupport': False}, {'extraGenerators': ['CodeBlocks', 'CodeLite', 'Sublime Text 2', 'Kate', 'Eclipse CDT4'], 'name': 'Unix Makefiles', 'platformSupport': False, 'toolsetSupport': False}, {'extraGenerators': ['CodeBlocks', 'CodeLite', 'Sublime Text 2', 'Kate', 'Eclipse CDT4'], 'name': 'Ninja', 'platformSupport': False, 'toolsetSupport': False}], 'serverMode': True, 'version': {'isDirty': False, 'major': 3, 'minor': 11, 'patch': 2, 'string': '3.11.2', 'suffix': ''}}, 'checkSystemVars': False, 'cookie': 'temp', 'debugOutput': False, 'extraGenerator': '', 'generator': 'Ninja', 'inReplyTo': 'globalSettings', 'sourceDirectory': '/home/umogslayer/zenith-cmake/zenith-engine/Src/CMakeEntryPoint', 'trace': False, 'traceExpand': False, 'type': 'rep

Balalaika
Balalaika2
Message read: len=1: 

Message read: len=23: [== "CMake Server" ==[

Message read: len=924: {"buildDirectory":"/home/umogslayer/zenith-cmake/temp_build","capabilities":{"generators":[{"extraGenerators":[],"name":"Watcom WMake","platformSupport":false,"toolsetSupport":false},{"extraGenerators":["CodeBlocks","CodeLite","Sublime Text 2","Kate","Eclipse CDT4"],"name":"Unix Makefiles","platformSupport":false,"toolsetSupport":false},{"extraGenerators":["CodeBlocks","CodeLite","Sublime Text 2","Kate","Eclipse CDT4"],"name":"Ninja","platformSupport":false,"toolsetSupport":false}],"serverMode":true,"version":{"isDirty":false,"major":3,"minor":11,"patch":2,"string":"3.11.2","suffix":""}},"checkSystemVars":false,"cookie":"temp","debugOutput":false,"extraGenerator":"","generator":"Ninja","inReplyTo":"globalSettings","sourceDirectory":"/home/umogslayer/zenith-cmake/zenith-engine/Src/CMakeEntryPoint","trace":false,"traceExpand":false,"type":"reply","warnUninitialized":false,"war

In [1]:
loop.run_until_complete(configure())

NameError: name 'loop' is not defined